In [1]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import dataLoader as dl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import util as ut

#from Inner_Speech_Dataset.Plotting.ERPs import 
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Select_time_window, Transform_for_classificator, Split_trial_in_time
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Calculate_power_windowed
from Inner_Speech_Dataset.Python_Processing.Utilitys import picks_from_channels
from Inner_Speech_Dataset.Python_Processing.Data_processing import Average_in_frec

In [2]:


sampling_rate = 128
nr_of_datasets=1
data, labels = dl.load_multiple_datasets(nr_of_datasets=nr_of_datasets, sampling_rate=sampling_rate, t_min=2, t_max=3)

ch_names = ut.get_channelNames()

print(data.shape)
data_p =  ut.get_power_array(data[:,:128,:], sampling_rate, trialSplit=1).squeeze()
print(data_p.shape)
#print(data_p[:,:,1])

#Getting Freq Data 
nr_of_buckets = 5
buckets = ut.createFreqBuckets(data[:,:128,:], nr_of_buckets)
print("buckets")
print(buckets)
data_f = ut.data_into_freq_buckets(data[:,:128,:], nr_of_buckets, buckets)
#data_f = np.concatenate([dataf, np.zeros([data_f.shape[0], nr_of_datasets, data_f.shape[2]])], axis=1)

print(data_f.shape)


## Normalize data



Data shape: [trials x channels x samples]
(500, 128, 128)
Labels shape
(500, 4)
Final data shape
(100, 128, 128)
Final labels shape
(100,)
Up is 0.0 and Down is 1.0
(100, 128, 128)
(100, 128, 2)
buckets
[[ 0  3]
 [ 3  9]
 [ 9 19]
 [19 40]
 [40 63]]
(100, 128, 5)


In [3]:
print(data_f.shape)
print(data_p.shape)
print(labels.shape)
print(data_p[2,5])
data = np.concatenate([data_f, data_p, data], axis =2 )
print(data.shape)
data = keras.utils.normalize(data, axis=1, order=2)
print(data.shape)
#print(data[2,5])
#print(labels[2])
print(labels[:].shape)

(100, 128, 5)
(100, 128, 2)
(100, 2)
[1.01648499e-10 5.52805943e-11]
(100, 128, 135)
(100, 128, 135)
(100, 2)


In [4]:
#Add end of ekg with specific array for each person. 


In [5]:
#Splitting into training and test data
#print(labels)


order = np.arange(labels.shape[0])
np.random.shuffle(order)

temp_data = np.zeros(data.shape)
temp_labels = np.zeros(labels.shape)

for x in range(labels.shape[0]):
    i = order[x]
    
    temp_data[x] = data[i]
    temp_labels[x] = labels[i]

data = temp_data
labels = temp_labels

data_train, data_test = np.split(data, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
labels_train, labels_test = np.split(labels, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
print(labels_train.shape)
print(data_train.shape)
print(data_test.shape)




(75, 2)
(75, 128, 135)
(25, 128, 135)


In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()





eeg_model = tf.keras.Sequential([
    layers.Dropout(0.7, input_shape = (data_train.shape[1],data_train.shape[2])),

    layers.LocallyConnected1D(556, input_shape = (data_train.shape[1],data_train.shape[2]), 
     kernel_size=30, 
     padding="valid", strides=2,
     activation="relu"),
    
    layers.Dropout(0.7),
    
    layers.LocallyConnected1D(510, 
    kernel_size=20, 
    padding="valid", strides=1,
    activation="relu"),
    
    layers.Dropout(0.8),
    
    layers.Dense(units=200, activation="relu"),
    
    layers.Dropout(0.8),

    layers.Flatten(),

    layers.Dense(units=2, activation="softmax")


])
eeg_model.build()
eeg_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 128, 135)          0         
                                                                 
 locally_connected1d (Locall  (None, 50, 556)          112617800 
 yConnected1D)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 50, 556)           0         
                                                                 
 locally_connected1d_1 (Loca  (None, 31, 510)          175823010 
 llyConnected1D)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 31, 510)           0         
                                                                 
 dense (Dense)               (None, 31, 200)           1

In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#callback2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=2)

eeg_model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])
#data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])

data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])
#data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])

outputs = eeg_model.fit(data_train, labels_train, validation_split=0.2 ,callbacks=[callback], epochs=50) #

print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

Epoch 1/50
2/2 [==============================] - 5s 2s/step - loss: 0.7056 - accuracy: 0.3833 - val_loss: 0.6917 - val_accuracy: 0.5333
Epoch 2/50
2/2 [==============================] - 2s 1s/step - loss: 0.7120 - accuracy: 0.4500 - val_loss: 0.6919 - val_accuracy: 0.4667
Epoch 3/50
2/2 [==============================] - 2s 1s/step - loss: 0.6412 - accuracy: 0.6667 - val_loss: 0.6953 - val_accuracy: 0.5333
Epoch 4/50
2/2 [==============================] - 2s 1s/step - loss: 0.7091 - accuracy: 0.7000 - val_loss: 0.7063 - val_accuracy: 0.6000
Epoch 5/50
2/2 [==============================] - 2s 1s/step - loss: 0.5893 - accuracy: 0.7167 - val_loss: 0.7758 - val_accuracy: 0.6000
Epoch 6/50
2/2 [==============================] - 2s 1s/step - loss: 0.5606 - accuracy: 0.7500 - val_loss: 0.8007 - val_accuracy: 0.4667
Epoch 7/50
2/2 [==============================] - 2s 1s/step - loss: 0.6244 - accuracy: 0.7667 - val_loss: 0.9593 - val_accuracy: 0.4000
Epoch 8/50
2/2 [=========================

In [8]:

eeg_model.save('saved_model/my_model6')

INFO:tensorflow:Assets written to: saved_model/my_model6\assets


In [9]:
print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

result2 = []
print(result)
for res in result:
    x = np.maximum(res[0], res[1])
    result2.append(int(np.where(res == x)[0]))

result2 = np.array(result2)
print(result2)

Results
1/1 [==============================] - 0s 40ms/step
[[0.5503834  0.44961667]
 [0.55808246 0.4419175 ]
 [0.53748953 0.46251044]
 [0.5412001  0.45879987]
 [0.553485   0.446515  ]
 [0.5389121  0.46108785]
 [0.5191419  0.480858  ]
 [0.544229   0.45577103]
 [0.56119984 0.43880007]
 [0.5127008  0.48729923]
 [0.5005562  0.49944386]
 [0.5730843  0.4269157 ]
 [0.54295933 0.45704067]
 [0.5618551  0.4381449 ]
 [0.5176411  0.4823589 ]
 [0.5481321  0.45186788]
 [0.5289308  0.47106925]
 [0.57554406 0.42445588]
 [0.57793003 0.42206997]
 [0.55334973 0.4466503 ]
 [0.51741904 0.48258096]
 [0.5425336  0.45746642]
 [0.53369397 0.4663061 ]
 [0.502013   0.49798694]
 [0.51821274 0.4817873 ]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
